# 1. Initialisation

## 1.1. File System

In [1]:
from google.colab import drive
drive.mount("/content/drive")

! chmod -R 777 'drive/My Drive/Otago/F. The Thesis/SEmHuS'

% cd 'drive/My Drive/Otago/F. The Thesis/SEmHuS'

Mounted at /content/drive
/content/drive/My Drive/Otago/F. The Thesis/SEmHuS


## 1.2. Importing Libraries

In [2]:
# !pip install livelossplot
# !pip install pandas-profiling==2.7.1
# !pip uninstall -y h5py
!pip install 'h5py==2.10.0' --force-reinstall

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 15.7 MB 30.3 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is inc

In [3]:
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport

import time
import re
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import nltk
import csv
import ast
import pickle
import itertools
import random
from scipy.spatial import distance

In [4]:
%tensorflow_version 1.x
import tensorflow as tf
from keras import Model
from keras.backend.tensorflow_backend import set_session
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn import metrics
from keras.preprocessing.sequence import pad_sequences

# from sklearn import preprocessing

# from keras.preprocessing.text import one_hot
# from tensorflow.keras.preprocessing.text import Tokenizer
# from keras.utils import to_categorical
# from keras import Input
# from keras.layers import Dense, Concatenate, BatchNormalization
# from keras.layers import Flatten, Dot
# from keras.layers import LSTM, GRU, Dropout
# from keras.layers import Lambda
# from keras.layers import Activation
# from keras.layers.embeddings import Embedding
import keras.backend as K
from keras.losses import mse

TensorFlow 1.x selected.


Using TensorFlow backend.


In [5]:
from keras.models import load_model

In [6]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

In [7]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
sys.path.insert(0, './Libraries')
import preprocess

## 1.3. GPU Initialisation

In [9]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



## 1.4. Global Variables

In [10]:
# model_name = 'RIVEC300_'
model_name = 'RIVEC300_SMALL_'
max_word = 3500
num_ent = 100
alpha = -1.5

In [11]:
Use_Word2Vec = True

## 1.5. Model Reloading

In [12]:
def custom_loss_func(first_pos, second_pos):
    def custom_loss(y_true, y_pred):
        first_idx = K.argmax(K.abs(first_pos), axis = -1)
        second_idx = K.argmax(K.abs(second_pos), axis = -1)
        
        first_max = tf.gather(first_pos, first_idx, axis = 1)
        second_max = tf.gather(second_pos, second_idx, axis = 1)

        first_second = K.mean(mse(first_max, second_max)) * 0.1
        first_first = K.binary_crossentropy(y_true, y_pred)

        return mse(first_second, first_first)
    custom_loss.__name__ = 'custom_loss'
    return custom_loss

In [13]:
# if Use_Word2Vec:
#     word_vectors = KeyedVectors.load_word2vec_format('./Pretrained/GoogleNews-vectors-negative300.bin', binary=True)

In [14]:
# if Use_Word2Vec: w2v_ext = 'With_Word2Vec'
# else: w2v_ext = 'Without_Word2Vec'

In [15]:
modelInputFile = f'./Model/{model_name}SEmHus_Word_Embedding_Model.h5'
model = load_model(modelInputFile)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



## 1.6. Tokenizers Reloading

In [16]:
with open(f'./Tokenizer/{model_name}text.pickle', 'rb') as handle:
    texts_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}actor.pickle', 'rb') as handle:
    labActor_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}agency.pickle', 'rb') as handle:
    labAgency_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}sector.pickle', 'rb') as handle:
    labSector_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}place.pickle', 'rb') as handle:
    labPlace_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}year.pickle', 'rb') as handle:
    labYear_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}month.pickle', 'rb') as handle:
    labMonth_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}reason.pickle', 'rb') as handle:
    labReason_tok = pickle.load(handle)

with open(f'./Tokenizer/{model_name}goal.pickle', 'rb') as handle:
    labGoal_tok = pickle.load(handle)

# 2. Data Preparation

In [17]:
def clean_alpha(text):
    text = ' '.join([w for w in str(text).split() if w.isalpha()])
    return text

## 2.1. Input and Output

In [18]:
# normInputFile = './Inputs/SEmHus_Enriched_Normalised_Dataset.csv'
normInputFile = f'./Inputs/{model_name}SEmHus_Enriched_Normalised_Dataset_Test.csv'
normDF = pd.read_csv(normInputFile)

In [19]:
normDF = normDF.loc[(normDF['country_name'] != "['World']") & (normDF['disaster_type'] != "['Other']") & (normDF['source_type'] != "['Media']")]

In [20]:
docs = normDF['text'].tolist()
labActor = [eval(v) for v in normDF['source_name'].tolist()]
labAgency = [eval(v) for v in normDF['source_type'].tolist()]
labSector = [eval(v) for v in normDF['theme'].tolist()]
labPlace = [eval(v) for v in normDF['country_name'].tolist()]
labYear = [eval(v) for v in normDF['year_created'].tolist()]
labMonth = [eval(v) for v in normDF['month_created'].tolist()]
labReason = [eval(v) for v in normDF['disaster_type'].tolist()]
labGoal = [eval(v) for v in normDF['development_goal'].tolist()]

In [21]:
# del normDF

In [22]:
labReason_tok.classes_

array(['Cold Wave', 'Drought', 'Earthquake', 'Epidemic',
       'Extratropical Cyclone', 'Fire', 'Flash Flood', 'Flood',
       'Heat Wave', 'Insect Infestation', 'Land Slide', 'Mud Slide',
       'Other', 'Severe Local Storm', 'Snow Avalanche', 'Storm Surge',
       'Technological Disaster', 'Tropical Cyclone', 'Tsunami', 'Volcano',
       'Wild Fire'], dtype=object)

In [23]:
# lbl_sec = labSector_tok.classes_.astype('U')
# np.savetxt('./Lists/sector_list.csv', lbl_sec, delimiter=',', fmt='%s')

In [24]:
# lbl_sec = labReason_tok.classes_.astype('U')
# np.savetxt('./Lists/reason_list.csv', lbl_sec, delimiter=',', fmt='%s')

In [25]:
docs = [' '.join(str(d).lower().split()[:max_word]) for d in docs]

In [26]:
len(labActor)

5280

In [27]:
padding_len = model.layers[0].get_output_at(0).get_shape().as_list()[1]

In [28]:
padding_len

100

## 2.2. Import Dictionaries

In [29]:
list_encoded_text = f'./Dictionary/{model_name}list_encoded_text.csv'
word_embed_dict = f'./Dictionary/{model_name}word_embed_dict.csv'

# 3. Class Embedding

## 3.1. Semantic Similarity Setup

In [30]:
# from sklearn.metrics.pairwise import cosine_similarity

# # Vectors
# vec_a = [1, 2, 3, 4, 5]
# vec_b = [1, 3, 5, 7, 9]

# # Dot and norm
# dot = sum(a*b for a, b in zip(vec_a, vec_b))
# norm_a = sum(a*a for a in vec_a) ** 0.5
# norm_b = sum(b*b for b in vec_b) ** 0.5

# # Cosine similarity
# cos_sim = dot / (norm_a*norm_b)

# # Results
# print('My version:', cos_sim)
# print('Scikit-Learn:', cosine_similarity([vec_a], [vec_b]))

In [31]:
def cosine(vec1, vec2):
    vec1 = np.array(vec1).tolist()
    vec2 = np.array(vec2).tolist()
    dot = sum(a*b for a, b in zip(vec1, vec2))
    norm_a = sum(a*a for a in vec1) ** 0.5
    norm_b = sum(b*b for b in vec2) ** 0.5
    cos_sim = dot / (norm_a*norm_b)
    return cos_sim

In [32]:
# num_word = 20

In [33]:
agency_weight = model.get_layer('outAgency').get_weights()[0]
agency_transpose = np.asarray(np.transpose(agency_weight))
agency_embed = {w: agency_transpose[idx] for idx, w in enumerate(labAgency_tok.classes_)}

actor_weight = model.get_layer('outActor').get_weights()[0]
actor_transpose = np.asarray(np.transpose(actor_weight))
actor_embed = {w: actor_transpose[idx] for idx, w in enumerate(labActor_tok.classes_)}

goal_weight = model.get_layer('outGoal').get_weights()[0]
goal_transpose = np.asarray(np.transpose(goal_weight))
goal_embed = {w: goal_transpose[idx] for idx, w in enumerate(labGoal_tok.classes_)}

sector_weight = model.get_layer('outSector').get_weights()[0]
sector_transpose = np.asarray(np.transpose(sector_weight))
sector_embed = {w: sector_transpose[idx] for idx, w in enumerate(labSector_tok.classes_)}

place_weight = model.get_layer('outPlace').get_weights()[0]
place_transpose = np.asarray(np.transpose(place_weight))
place_embed = {w: place_transpose[idx] for idx, w in enumerate(labPlace_tok.classes_)}

year_weight = model.get_layer('outYear').get_weights()[0]
year_transpose = np.asarray(np.transpose(year_weight))
year_embed = {w: year_transpose[idx] for idx, w in enumerate(labYear_tok.classes_)}

month_weight = model.get_layer('outMonth').get_weights()[0]
month_transpose = np.asarray(np.transpose(month_weight))
month_embed = {w: month_transpose[idx] for idx, w in enumerate(labMonth_tok.classes_)}

reason_weight = model.get_layer('outReason').get_weights()[0]
reason_transpose = np.asarray(np.transpose(reason_weight))
reason_embed = {w: reason_transpose[idx] for idx, w in enumerate(labReason_tok.classes_)}

In [34]:
# def nearest_label(group, entity):
#     if group == '*':
#         new_df = class_df
#     else:
#         new_df = class_df.loc[class_df['Class'] == group]

#     vec1 = new_df.loc[class_df['Entity'] == entity]
#     vec1 = vec1.iloc[0]['Vector']
#     dist = []
#     # for cls, ent, vec2 in class_data:
#     for cls, ent, vec2 in class_df.values.tolist():
#         d = distance.cosine(vec1, vec2)
#         if d > 0:
#             dist.append((round(distance.cosine(vec1, vec2), 4), ent, cls))
#     dist.sort()
#     dist = dist[:20]
#     return dist

In [35]:
# def nearest_class(first, second, third, num_ent):
#     if first == 'Agency': class_list = agency_embed.items()
#     if first == 'Actor': class_list = actor_embed.items()
#     if first == 'Goal': class_list = goal_embed.items()
#     if first == 'Sector': class_list = sector_embed.items()
#     if first == 'Place': class_list = place_embed.items()
#     if first == 'Year': class_list = year_embed.items()
#     if first == 'Month': class_list = month_embed.items()
#     if first == 'Reason': class_list = reason_embed.items()

#     if second == 'Agency': vec1 = agency_embed[third]
#     if second == 'Actor': vec1 = actor_embed[third]
#     if second == 'Goal': vec1 = goal_embed[third]
#     if second == 'Sector': vec1 = sector_embed[third]
#     if second == 'Place': vec1 = place_embed[third]
#     if second == 'Year': vec1 = year_embed[third]
#     if second == 'Month': vec1 = month_embed[third]
#     if second == 'Reason': vec1 = reason_embed[third]

#     dist = []
#     for word, vec2 in class_list:
#         d = distance.cosine(vec1, vec2)
#         if d > 0:
#             #dist.append((np.linalg.norm(vec1 - vec2), word))
#             dist.append((round(d, 4), word))
#     dist.sort()
#     dist = dist[:num_ent]
#     return dist

## 3.2. Vector Space Export

In [36]:
def normalize(word_vec):
    norm=np.linalg.norm(word_vec)
    if norm == 0:
       return word_vec
    return word_vec/norm

In [37]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)

In [38]:
cols = ['Class', 'Entity', 'Vector', 'X', 'Y']
rows = []

ent = []
nam = []
vec = []

for key, value in agency_embed.items():
    ent.append('Agency')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in actor_embed.items():
    ent.append('Actor')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in goal_embed.items():
    ent.append('Goal')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in sector_embed.items():
    ent.append('Sector')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in place_embed.items():
    ent.append('Place')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in year_embed.items():
    ent.append('Year')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in month_embed.items():
    ent.append('Month')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

for key, value in reason_embed.items():
    ent.append('Reason')
    nam.append(key)
    normed = normalize(list(value))
    vec.append(normed)

reduced_vector = tsne.fit_transform(vec)
X_dim = reduced_vector[:,0]
Y_dim = reduced_vector[:,1]

In [39]:
rows = np.array([ent, nam, vec, X_dim, Y_dim])

rows_t = rows.T 

class_df = pd.DataFrame(data=rows_t, columns = cols)
class_df.to_csv(f'./Outputs/{model_name}Humanitarian_Class_Vector_Space.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [40]:
# class_data = class_df.values.tolist()

## 3.3. Entity Distance Export

In [41]:
file_name = f'./Outputs/{model_name}Humanitarian_Class_Distance_Space.csv'

if os.path.isfile(file_name):
    dist_df = pd.read_csv(file_name)
else:
    cols = ['Source_Class', 'Source_Entity', 'Target_Class', 'Target_Entity', 'Distance']
    rows = []

    for sidx, source in class_df.iterrows():
        for tidx, target in class_df.iterrows():

            if (source['Class'] == target['Class']) and (source['Entity'] == target['Entity']): continue
            
            # dist = distance.euclidean(source['Vector'], target['Vector'])
            dist = distance.cosine(source['Vector'], target['Vector'])
            rows.append([source['Class'], source['Entity'], target['Class'], target['Entity'], dist])

    dist_df = pd.DataFrame(data=rows, columns = cols)
    dist_df.to_csv(file_name)

In [42]:
# file_name = f'./Outputs/{model_name}Humanitarian_Class_Distance_Space.csv'

# cols = ['Source_Class', 'Source_Entity', 'Target_Class', 'Target_Entity', 'Distance']
# rows = []

# for sidx, source in class_df.iterrows():
#     for tidx, target in class_df.iterrows():

#         if (source['Class'] == target['Class']) and (source['Entity'] == target['Entity']): continue
        
#         # dist = distance.euclidean(source['Vector'], target['Vector'])
#         dist = distance.cosine(source['Vector'], target['Vector'])
#         rows.append([source['Class'], source['Entity'], target['Class'], target['Entity'], dist])

# dist_df = pd.DataFrame(data=rows, columns = cols)
# dist_df.to_csv(file_name)

In [43]:
def merge(list1, list2):
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
    return merged_list

In [44]:
def nearest_class(first, second, third, near):

    class_list = dist_df[(dist_df['Source_Class'] == first) & (dist_df['Target_Class'] == second) & (dist_df['Target_Entity'] == third) ]

    dist = []

    # nam = []
    # dis = []

    for idx, row in class_list.iterrows():
        d = row[5]
        # d = row[4]
        if d > 0:
            #dist.append((np.linalg.norm(vec1 - vec2), word))
            dist.append((round(d, 4), row['Source_Entity']))
            # nam.append(row['Source_Entity'])
            # dis.append(d)
    
    # dis = normalize(dis)
    # dist = merge(dis, nam)
    
    dist.sort()
    dist = dist[:near]

    return dist

In [45]:
def farest_class(first, second, third, near):

    class_list = dist_df[(dist_df['Source_Class'] == first) & (dist_df['Target_Class'] == second) & (dist_df['Target_Entity'] == third) ]


    dist = []

    # nam = []
    # dis = []
    
    for idx, row in class_list.iterrows():
        d = row[5]
        # d = row[4]
        if d > 0:
            #dist.append((np.linalg.norm(vec1 - vec2), word))
            dist.append((round(d, 4), row['Source_Entity']))
            # nam.append(row['Source_Entity'])
            # dis.append(d)
    
    # dis = normalize(dis)
    # dist = merge(dis, nam)

    dist.sort()
    dist = dist[-near:]

    return dist

In [46]:
def find_stat(cls):
    if cls == '*':
        temp_df = dist_df[(dist_df['Distance'] > 0)]
    else:
        temp_df = dist_df[(dist_df['Source_Class'] == cls) & (dist_df['Distance'] > 0)]

    dist_lst = temp_df['Distance'].values

    sd = np.std(dist_lst)
    me = np.mean(dist_lst)
    mx = np.max(dist_lst)
    mn = np.min(dist_lst)
    cs = (mx - mn) / sd

    return sd, me, mx, mn, cs

In [47]:
def find_rank(cls, val, alpha):
    
    stat = find_stat(cls)
    
    stdev = stat[0]
    mean = stat[1]
    # measure = mean - (alpha * stdev)
    measure = (val - mean) / stdev

    # if alpha < measure: rank = 'Accepted'
    # if alpha >= measure: rank = 'Rejected'

    if alpha >= measure: rank = 'Accepted'
    if alpha < measure: rank = 'Rejected'

    return rank

In [48]:
# class_df = dist_df[(dist_df['Source_Class'] == 'Place') & (dist_df['Distance'] > 0)]
# # class_df = dist_df[(dist_df['Distance'] > 0)]

# dist_lst = class_df['Distance'].values

cls = '*'
stat = find_stat(cls)

print('Standard Deviation:', round(stat[0], 6))
print('Mean Distance:', round(stat[1], 6))
print('Maximum Distance:', round(stat[2], 6))
print('Minimum Distance :', round(stat[3], 6))
print('Number of CS :', round(stat[4], 6))

Standard Deviation: 0.10367
Mean Distance: 0.953508
Maximum Distance: 1.444925
Minimum Distance : 0.033215
Number of CS : 13.617288


# 4. Result Validation

In [49]:
# var1, var2 = 'Year', 'Year'

# if var2 == 'Agency': rand_class = labAgency_tok.classes_[random.randint(0, len(labAgency_tok.classes_)-1)]
# if var2 == 'Actor': rand_class = labActor_tok.classes_[random.randint(0, len(labActor_tok.classes_)-1)]
# if var2 == 'Goal': rand_class = labGoal_tok.classes_[random.randint(0, len(labGoal_tok.classes_)-1)]
# if var2 == 'Sector': rand_class = labSector_tok.classes_[random.randint(0, len(labSector_tok.classes_)-1)]
# if var2 == 'Place': rand_class = labPlace_tok.classes_[random.randint(0, len(labPlace_tok.classes_)-1)]
# if var2 == 'Year': rand_class = labYear_tok.classes_[random.randint(0, len(labYear_tok.classes_)-1)]
# if var2 == 'Month': rand_class = labMonth_tok.classes_[random.randint(0, len(labMonth_tok.classes_)-1)]
# if var2 == 'Reason': rand_class = labReason_tok.classes_[random.randint(0, len(labReason_tok.classes_)-1)]

# rand_class = '1996'

# ents = nearest_class(var1, var2, rand_class, num_ent)

# accepted_list = ''
# rejected_list = ''

# acc_ent = []
# rej_ent = []

# nearest_list = ''

# en, dt, sd, me = [], [], [], []

# # # cls = '*'
# cls = var1

# # alpha = -1.0

# stat = find_stat(cls)

# for ent in ents:
#     rank = find_rank(cls, ent[0], alpha)
#     # if rank == 'Accepted':
#     en.append(ent[1])
#     dt.append(ent[0])
#     sd.append(stat[0])
#     me.append(stat[1])
#     d = round(ent[0], 4)
#     z = round((ent[0] - stat[1]) / stat[0], 4)
#     if rank == 'Accepted':
#         accepted_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
#         acc_ent.append(ent[1])
#     else:
#         rejected_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
#         rej_ent.append(ent[1])
    
#     nearest_list += f'{ent[1]} ({z}); '
#     # nearest_list += f'{ent[1]} ({d}); '

# # nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'
# nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'

# print(nearest_title)
# print(nearest_list)

In [50]:
# break

## 4.1. Mapping Table

In [51]:
from sqlite3 import connect
conn = connect(':memory:')

In [52]:
sqlFile = './Inputs/EMDAT_1900-2021_NatDis.csv'
sqlDF = pd.read_csv(sqlFile)
sqlDF = sqlDF.loc[(sqlDF['Year'] >= 1995) & (sqlDF['Year'] <= 2016)]

In [53]:
years = sqlDF['Year'].unique()
years

array([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016])

In [54]:
places = sqlDF['Country'].unique()
places

array(['Azerbaijan', 'Burkina Faso', 'Bangladesh', 'Colombia', 'China',
       'Costa Rica', 'Afghanistan', 'Angola', 'Albania',
       'Netherlands Antilles', 'Argentina', 'Antigua and Barbuda',
       'Australia', 'Austria', 'Belgium', 'Benin', 'Bahamas', 'Belarus',
       'Belize', 'Brazil', 'Barbados', 'Botswana',
       'Central African Republic', 'Canada', 'Chile', 'Côte d’Ivoire',
       'Cabo Verde', 'Cuba', 'Cyprus', 'Germany', 'Azores Islands',
       'Bolivia (Plurinational State of)', 'Cameroon', 'Czech Republic',
       'Armenia', 'Burundi', 'Bulgaria', 'Switzerland', 'Congo',
       'Cook Islands', 'Brunei Darussalam', 'Sri Lanka', 'Japan',
       'Philippines', 'Indonesia', 'Honduras', 'India', 'Nepal', 'France',
       'United Kingdom of Great Britain and Northern Ireland',
       'Korea (the Republic of)', 'Luxembourg', 'Netherlands', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'Eritrea',
       'Spain', 'Ethiopia', 'Georgia', 'Ghana', 'Gua

In [55]:
sqlDF['Disaster_Type'].unique()

array(['Flood', 'Drought', 'Earthquake', 'Storm', 'Landslide', 'Epidemic',
       'Extreme temperature', 'Wildfire', 'Insect infestation',
       'Volcanic activity', 'Mass movement (dry)', 'Impact',
       'Animal accident'], dtype=object)

In [56]:
sqlDF['Disaster_Subtype'].unique()

array(['Riverine flood', 'Drought', nan, 'Ground movement',
       'Convective storm', 'Tropical cyclone', 'Landslide',
       'Bacterial disease', 'Flash flood', 'Cold wave', 'Heat wave',
       'Coastal flood', 'Parasitic disease', 'Viral disease', 'Mudslide',
       'Forest fire', 'Ash fall', 'Avalanche',
       'Land fire (Brush, Bush, Pasture)', 'Locust', 'Tsunami',
       'Extra-tropical storm', 'Subsidence', 'Severe winter conditions',
       'Rockfall', 'Lava flow'], dtype=object)

In [57]:
sqlDF['Associated_Dis'].unique()

array(['Slide (land, mud, snow, rock)', 'Food shortage',
       'Tsunami/Tidal wave', 'Hail', 'Flood', nan,
       'Broken Dam/Burst bank', 'Rain', 'Fire', 'Transport accident',
       'Surge', 'Wildfire', 'Crop failure', 'Heat wave', 'Cold wave',
       'Water shortage', 'Drought', 'Famine', 'Collapse', 'Snow/ice',
       'Earthquake', 'Pollution', 'Explosion', 'Avalanche (Snow, Debris)',
       'Storm', 'Lightening', 'Industrial accidents', 'Fog', 'Epidemic'],
      dtype=object)

In [58]:
sqlDF['Associated_Dis2'].unique()

array(['Tsunami/Tidal wave', 'Famine', nan,
       'Slide (land, mud, snow, rock)', 'Flood', 'Hail', 'Water shortage',
       'Transport accident', 'Food shortage', 'Surge', 'Explosion',
       'Rain', 'Drought', 'Broken Dam/Burst bank', 'Collapse',
       'Avalanche (Snow, Debris)', 'Wildfire', 'Snow/ice', 'Pollution',
       'Storm', 'Cold wave', 'Heat wave', 'Fire', 'Lightening',
       'Industrial accidents', 'Fog', 'Epidemic'], dtype=object)

In [59]:
# labReason_tok.classes_[4] = 'Cyclone'
# labReason_tok.classes_[6] = 'Flood'
# labReason_tok.classes_[13] = 'Storm'
# labReason_tok.classes_[15] = 'Storm'
# labReason_tok.classes_[17] = 'Cyclone'
# labReason_tok.classes_[11] = 'Land Slide'
# labReason_tok.classes_[20] = 'Fire'

In [60]:
labReason_tok.classes_

array(['Cold Wave', 'Drought', 'Earthquake', 'Epidemic',
       'Extratropical Cyclone', 'Fire', 'Flash Flood', 'Flood',
       'Heat Wave', 'Insect Infestation', 'Land Slide', 'Mud Slide',
       'Other', 'Severe Local Storm', 'Snow Avalanche', 'Storm Surge',
       'Technological Disaster', 'Tropical Cyclone', 'Tsunami', 'Volcano',
       'Wild Fire'], dtype=object)

In [61]:
sqlDF.loc[sqlDF.Disaster_Type == 'Volcanic activity', 'Disaster_Type'] = 'Volcano'
sqlDF.loc[sqlDF.Disaster_Type == 'Landslide', 'Disaster_Type'] = 'Land Slide'
sqlDF.loc[sqlDF.Disaster_Type == 'Wildfire', 'Disaster_Type'] = 'Wild Fire'
sqlDF.loc[sqlDF.Disaster_Type == 'Storm', 'Disaster_Type'] = 'Storm Surge'
sqlDF.loc[sqlDF.Disaster_Type == 'Insect infestation', 'Disaster_Type'] = 'Insect Infestation'
sqlDF.loc[sqlDF.Disaster_Type == 'Mass movement (dry)', 'Disaster_Type'] = 'Land Slide'

sqlDF.loc[sqlDF.Disaster_Subtype == 'Cold wave', 'Disaster_Type'] = 'Cold Wave'
sqlDF.loc[sqlDF.Disaster_Subtype == 'Heat wave', 'Disaster_Type'] = 'Heat Wave'
sqlDF.loc[sqlDF.Disaster_Subtype == 'Avalanche', 'Disaster_Type'] = 'Snow Avalanche'
sqlDF.loc[sqlDF.Disaster_Subtype == 'Mudslide', 'Disaster_Type'] = 'Mud Slide'
sqlDF.loc[sqlDF.Disaster_Subtype == 'Tropical cyclone', 'Disaster_Type'] = 'Tropical Cyclone'
sqlDF.loc[sqlDF.Disaster_Subtype == 'Rockfall', 'Disaster_Type'] = 'Land Slide'
sqlDF.loc[sqlDF.Disaster_Subtype == 'Severe winter conditions', 'Disaster_Type'] = 'Cold Wave'

# sqlDF.loc[sqlDF.Associated_Dis == 'Tsunami/Tidal wave', 'Disaster_Type'] = 'Tsunami'

In [62]:
# sqlDF

In [63]:
sqlDF.to_sql('Hum_Hist', conn)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [64]:
unwanted_list = ['Other', 'Technological Disaster']

In [65]:
var1, var2 = 'Reason', 'Place'

if var2 == 'Agency': rand_class = labAgency_tok.classes_[random.randint(0, len(labAgency_tok.classes_)-1)]
if var2 == 'Actor': rand_class = labActor_tok.classes_[random.randint(0, len(labActor_tok.classes_)-1)]
if var2 == 'Goal': rand_class = labGoal_tok.classes_[random.randint(0, len(labGoal_tok.classes_)-1)]
if var2 == 'Sector': rand_class = labSector_tok.classes_[random.randint(0, len(labSector_tok.classes_)-1)]
if var2 == 'Place': rand_class = labPlace_tok.classes_[random.randint(0, len(labPlace_tok.classes_)-1)]
if var2 == 'Year': rand_class = labYear_tok.classes_[random.randint(0, len(labYear_tok.classes_)-1)]
if var2 == 'Month': rand_class = labMonth_tok.classes_[random.randint(0, len(labMonth_tok.classes_)-1)]
if var2 == 'Reason': rand_class = labReason_tok.classes_[random.randint(0, len(labReason_tok.classes_)-1)]

# rand_class = labPlace_tok.classes_[random.randint(0, len(labPlace_tok.classes_)-1)]
# rand_class = 'Sudan'
# rand_class = 'United Kingdom of Great Britain and Northern Ireland'
rand_class = 'Cook Islands'

rand_year = labYear_tok.classes_[random.randint(0, len(labYear_tok.classes_)-1)]

ents = nearest_class(var1, var2, rand_class, num_ent)

accepted_list = ''
rejected_list = ''

acc_ent = []
rej_ent = []

en, dt, sd, me = [], [], [], []

# cls = '*'
cls = var1

alpha = -0.25

stat = find_stat(cls)

for ent in ents:
    if ent[1] in unwanted_list:
        # print(ent[1])
        continue
    rank = find_rank(cls, ent[0], alpha)
    # if rank == 'Accepted':
    en.append(ent[1])
    dt.append(ent[0])
    sd.append(stat[0])
    me.append(stat[1])
    d = round(ent[0], 4)
    z = round((ent[0] - stat[1]) / stat[0], 4)
    if rank == 'Accepted':
        accepted_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
        acc_ent.append(ent[1])
    else:
        rejected_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
        rej_ent.append(ent[1])
    # nearest_list += f'{ent[1]} ({z}); '
    # nearest_list += f'{ent[1]} ({d}); \n'

# nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'
nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'

print(nearest_title)
print('Accepted Results:')
print(accepted_list)
print('Rejected Results:')
print(rejected_list)

Nearest Reason(s) to Place [Cook Islands] are 19:

Accepted Results:
Tropical Cyclone [Distance=0.7466, Z-Score=-2.9208] -- [Accepted];
Storm Surge [Distance=0.7932, Z-Score=-2.3618] -- [Accepted];
Cold Wave [Distance=0.8476, Z-Score=-1.7091] -- [Accepted];
Tsunami [Distance=0.922, Z-Score=-0.8166] -- [Accepted];
Drought [Distance=0.9231, Z-Score=-0.8034] -- [Accepted];
Epidemic [Distance=0.9298, Z-Score=-0.723] -- [Accepted];
Wild Fire [Distance=0.9341, Z-Score=-0.6714] -- [Accepted];
Volcano [Distance=0.9559, Z-Score=-0.4099] -- [Accepted];
Insect Infestation [Distance=0.9662, Z-Score=-0.2863] -- [Accepted];

Rejected Results:
Severe Local Storm [Distance=0.9877, Z-Score=-0.0284] -- [Rejected];
Fire [Distance=0.9913, Z-Score=0.0148] -- [Rejected];
Extratropical Cyclone [Distance=1.0062, Z-Score=0.1936] -- [Rejected];
Mud Slide [Distance=1.0233, Z-Score=0.3987] -- [Rejected];
Heat Wave [Distance=1.0356, Z-Score=0.5463] -- [Rejected];
Snow Avalanche [Distance=1.0391, Z-Score=0.5882] --

In [66]:
# var1, var2 = 'Reason', 'Place'

# if var2 == 'Agency': rand_class = labAgency_tok.classes_[random.randint(0, len(labAgency_tok.classes_)-1)]
# if var2 == 'Actor': rand_class = labActor_tok.classes_[random.randint(0, len(labActor_tok.classes_)-1)]
# if var2 == 'Goal': rand_class = labGoal_tok.classes_[random.randint(0, len(labGoal_tok.classes_)-1)]
# if var2 == 'Sector': rand_class = labSector_tok.classes_[random.randint(0, len(labSector_tok.classes_)-1)]
# if var2 == 'Place': rand_class = labPlace_tok.classes_[random.randint(0, len(labPlace_tok.classes_)-1)]
# if var2 == 'Year': rand_class = labYear_tok.classes_[random.randint(0, len(labYear_tok.classes_)-1)]
# if var2 == 'Month': rand_class = labMonth_tok.classes_[random.randint(0, len(labMonth_tok.classes_)-1)]
# if var2 == 'Reason': rand_class = labReason_tok.classes_[random.randint(0, len(labReason_tok.classes_)-1)]

# rand_class = 'Sudan'

# acc_ent = nearest_class(var1, var2, rand_class, num_ent)
# rej_ent = farest_class(var1, var2, rand_class, num_ent)

# acc_ent = [e for (d, e) in acc_ent]
# rej_ent = [e for (d, e) in rej_ent]

# acc_ent, rej_ent

## 4.2. Using Benchmarks

In [67]:
if var1 == 'Place': source_col = 'Country'
if var1 == 'Year': source_col = 'Year'
if var1 == 'Reason': source_col1 = 'Disaster_Type'
if var1 == 'Reason': source_col2 = 'Disaster_Subtype'
if var1 == 'Reason': source_col2 = 'Disaster_Subtype'

if var2 == 'Place': target_col = 'Country'
if var2 == 'Year': target_col = 'Year'
if var2 == 'Reason': target_col1 = 'Disaster_Type'
if var2 == 'Reason': target_col2 = 'Disaster_Subtype'

In [68]:
def sql_return(source_col, target_col, rand_class, rand_year):
    tempDF = pd.read_sql(f'''SELECT {source_col}, COUNT(Dis_No) AS Freq
                            FROM Hum_Hist
                            WHERE {target_col} LIKE "%{rand_class}%" AND Year = {rand_year}
                            GROUP BY {source_col}
                            ORDER BY Freq Desc''', conn)
    return tempDF

In [69]:
freq_title = f'Nearest {var1}(s) to {var2} [{rand_class}] and Year [{rand_year}] are:\n'

OneDF = sql_return(source_col1, target_col, rand_class, rand_year)

pd.options.display.max_colwidth = 0

print(freq_title)

OneDF

Nearest Reason(s) to Place [Cook Islands] and Year [2006] are:



,Disaster_Type,Freq


In [70]:
freq_title = f'Nearest {var1}(s) to {var2} [{rand_class}] and Year [{rand_year}] are:\n'

TwoDF = sql_return(source_col2, target_col, rand_class, rand_year)

pd.options.display.max_colwidth = 0

print(freq_title)

TwoDF

Nearest Reason(s) to Place [Cook Islands] and Year [2006] are:



,Disaster_Subtype,Freq


In [71]:
inf = []
pre = []

for e in acc_ent:
    x = e.split()
    new = 0
    for y in x:
        if OneDF[source_col1].str.contains(y).any() or TwoDF[source_col2].str.contains(y).any():
            new = 1
        # else:
        #     new = 0

    if new == 1:
        inf.append(e)
    else:
        pre.append(e)

print(freq_title)
print('All Matched: ', inf)
print('Not Matched: ', pre)
print('')

Nearest Reason(s) to Place [Cook Islands] and Year [2006] are:

All Matched:  []
Not Matched:  ['Tropical Cyclone', 'Storm Surge', 'Cold Wave', 'Tsunami', 'Drought', 'Epidemic', 'Wild Fire', 'Volcano', 'Insect Infestation']



## 4.3. Clustering the Results

In [72]:
# from sklearn.cluster import KMeans

In [73]:
# ent_vec = []

# for e in en:
#     rec = class_df.loc[class_df['Entity'] == e]
#     ent_vec.append(rec['Vector'].tolist()[0])

In [74]:
# kmeans = KMeans(n_clusters=2, random_state=0).fit(ent_vec)
# clust = kmeans.labels_
# # kmeans.cluster_centers_ 
# # USE THIS FUNCTION

In [75]:
# clust

In [76]:
# rel = []
# irr = []

# for idx, e in enumerate(acc_ent):
#     if clust[idx] == clust[0]:
#         rel.append(e)
#     else:
#         irr.append(e)

# print(freq_title)
# print('Most Relevant:  ', rel)
# print('Least Relevant: ', irr)
# print('')

## 4.4. Export Results

In [77]:
listEarthquake = ['Earthquake', 'Tsunami']
listVolcano = ['Volcano']
listStorm = ['Severe Local Storm', 'Storm Surge', 'Tropical Cyclone', 'Extratropical Cyclone']
listTemperature = ['Heat Wave', 'Cold Wave']
listFlood = ['Flash Flood', 'Flood']
listLandslide = ['Mud Slide', 'Land Slide', 'Snow Avalanche']
listDrought = ['Drought']
listFire = ['Fire', 'Wild Fire']
listEpidemic = ['Epidemic']
listInsect = ['Insect Infestation']

In [78]:
New_Reason = ['Earthquake', 'Volcano', 'Storm', 'Extreme Temperature', 'Flood', 'Landslide', 'Drought', 'Fire', 'Epidemic', 'Insect']

In [79]:
def dis_cls(lst):
    
    disType = []

    for l in lst:
        if l in listEarthquake: disType.append('Earthquake')
        if l in listVolcano: disType.append('Volcano')
        if l in listStorm: disType.append('Storm')
        if l in listTemperature: disType.append('Extreme Temperature')
        if l in listFlood: disType.append('Flood')
        if l in listLandslide: disType.append('Landslide')
        if l in listDrought: disType.append('Drought')
        if l in listFire: disType.append('Fire')
        if l in listEpidemic: disType.append('Epidemic')
        if l in listInsect: disType.append('Insect')

    disType = list(set(disType))

    return disType

In [80]:
# listGeophysical = ['Earthquake', 'Tsunami', 'Volcano']
# listMeteorological = ['Heat Wave', 'Cold Wave', 'Severe Local Storm', 'Storm Surge', 'Tropical Cyclone', 'Extratropical Cyclone']
# listHydrological = ['Flash Flood', 'Flood', 'Mud Slide', 'Land Slide', 'Snow Avalanche']
# listClimatological = ['Drought', 'Fire', 'Wild Fire']
# listBiological = ['Epidemic', 'Insect Infestation']

In [81]:
# def dis_cls(lst):
    
#     disType = []

#     for l in lst:
#         if l in listGeophysical: disType.append('Geophysical')
#         if l in listMeteorological: disType.append('Meteorological')
#         if l in listHydrological: disType.append('Hydrological')
#         if l in listClimatological: disType.append('Climatological')
#         if l in listBiological: disType.append('Biological')

#     disType = list(set(disType))

#     return disType

In [82]:
def SM_Reasons(plc):
    
    var1, var2 = 'Reason', 'Place'

    rand_class = plc

    ents = nearest_class(var1, var2, rand_class, num_ent)

    accepted_list = ''
    rejected_list = ''

    acc_ent = []
    rej_ent = []

    en, dt, sd, me = [], [], [], []

    # cls = '*'
    cls = var1

    # alpha = -1.0

    stat = find_stat(cls)

    for ent in ents:
        if ent[1] in unwanted_list:
            continue
        rank = find_rank(cls, ent[0], alpha)
        # if rank == 'Accepted':
        en.append(ent[1])
        dt.append(ent[0])
        sd.append(stat[0])
        me.append(stat[1])
        d = round(ent[0], 4)
        z = round((ent[0] - stat[1]) / stat[0], 4)
        if rank == 'Accepted':
            accepted_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
            acc_ent.append(ent[1])
        else:
            rejected_list += f'{ent[1]} [Distance={d}, Z-Score={z}] -- [{rank}];\n'
            rej_ent.append(ent[1])
        # nearest_list += f'{ent[1]} ({z}); '
        # nearest_list += f'{ent[1]} ({d}); \n'

    # nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'
    nearest_title = f'Nearest {var1}(s) to {var2} [{rand_class}] are {len(en)}:\n'
    return acc_ent, rej_ent

In [83]:
def EM_Reasons(plc, year, results):

    accepted, rejected = dis_cls(results[0]), dis_cls(results[1])

    rej_temp = []

    for r in rejected:
        if r not in accepted:
            rej_temp.append(r)

    rejected = rej_temp

    tp = []
    fp = []

    tn = []
    fn = []

    OneDF = sql_return(source_col1, target_col, plc, year)
    twoDF = sql_return(source_col2, target_col, plc, year)

    OneDF = dis_cls(OneDF[source_col1].values.tolist())
    twoDF = dis_cls(twoDF[source_col2].values.tolist())

    bothDF = OneDF + twoDF
    bothDF = list(set(bothDF))

    # print(OneDF)
    # print(twoDF)
    # print(bothDF, '\n')

    if len(bothDF) >= 1:

        for e in accepted:
            if e in bothDF:
                tp.append(e)
            else:
                fp.append(e)

        for e in rejected:
            if e in bothDF:
                fn.append(e)
            else:
                tn.append(e)

    return tp, fp, tn, fn, bothDF

## Signle Country

In [84]:
plc = labPlace_tok.classes_[random.randint(0, len(labPlace_tok.classes_)-1)]
# plc = 'Sudan'
# plc = 'United Kingdom of Great Britain and Northern Ireland'
# plc = rand_class
# plc = 'Romania'

year = rand_year

if plc not in places:
    print('Sorry, country not found!\n')
else:
    print(f'Evalaution of {plc}:')

alpha = -1.0

SM_RE = SM_Reasons(plc)
EM_RE = EM_Reasons(plc, year, SM_RE)

act_sem = dis_cls(SM_RE[0])
act_emd = EM_RE[4]

y_act = []
y_pre = []

for r in New_Reason:
    if r in act_emd and r in act_sem:
        y_act.append(1)
        y_pre.append(1)
    elif r in act_emd and r not in act_sem:
        y_act.append(1)
        y_pre.append(0)
    elif r not in act_emd and r in act_sem:
        y_act.append(0)
        y_pre.append(1)
    else:
        y_act.append(0)
        y_pre.append(0)

# for r in New_Reason:
#     if r in act_sem and r in act_emd:
#         y_act.append(r)
#         y_pre.append(r)
#     elif r in act_sem and r not in act_emd:
#         y_act.append(r)
#         y_pre.append('None')
#     elif r not in act_sem and r in act_emd:
#         y_act.append('None')
#         y_pre.append(r)
#     else:
#         y_act.append('None')
#         y_pre.append('None')

# cls_res = score(y_act, y_pre, average='weighted', zero_division=0)

# cls_pre, cls_rec, cls_f1, cls_sup = cls_res

cls_pre = metrics.precision_score(y_act, y_pre)
cls_rec = metrics.recall_score(y_act, y_pre)
cls_f1 = metrics.f1_score(y_act, y_pre)
cls_hmm = metrics.hamming_loss(y_act, y_pre)

temp_TP = EM_RE[0]
temp_FP = EM_RE[1]
temp_TN = EM_RE[2]
temp_FN = EM_RE[3]

count_TP = len(temp_TP)
count_FP = len(temp_FP)
count_TN = len(temp_TN)
count_FN = len(temp_FN)

if (count_TP + count_FP + count_FN + count_TN) >= 1:

    temp_acc = (count_TP + count_TN) / (count_TP + count_FP + count_FN + count_TN)

    if count_TP + count_FP == 0:
        temp_pre = 0
    else:
        temp_pre = count_TP / (count_TP + count_FP)

    if count_TP + count_FN == 0:
        temp_rec = 0
    else:
        temp_rec = count_TP / (count_TP + count_FN)

    if (temp_rec + temp_pre) == 0:
        temp_f1 = 0
    else:
        temp_f1 = 2 * (temp_rec * temp_pre) / (temp_rec + temp_pre)

print('\r')
print(f'All Classes ({len(New_Reason)})\t', New_Reason)

print('\r')
print(f'Result (EM-DAT) ({len(act_emd)})\t', act_emd)
print(f'Result (SEmHuS) ({len(act_sem)})\t', act_sem)

print('\r')
print(f'Y-Act (EM-DAT) ({len(y_act)})\t', y_act)
print(f'Y-Pre (SEmHuS) ({len(y_pre)})\t', y_pre)

print('\r')
print(plc, f'Positive {SM_RE[0]}')
print(plc, f'Negative {SM_RE[1]}')

print('\r')
print(f'True Positive  ({len(temp_TP)})\t', temp_TP)
print(f'False Positive ({len(temp_FP)})\t', temp_FP)

print('\r')
print(f'True Negative  ({len(temp_TN)})\t', temp_TN)
print(f'False Negative ({len(temp_FN)})\t', temp_FN)

# print('\r')
# print(f'Label Accuracy  \t', round(temp_acc * 100, 2), '%')

print('\r')
print(f'Label Precision  \t', round(temp_pre * 100, 2), '%')
print(f'Label Recall  \t\t', round(temp_rec * 100, 2), '%')
print(f'Label F1_score  \t', round(temp_f1 * 100, 2), '%')

print('\r')
print(f'Multi Precision  \t', round(cls_pre * 100, 2), '%')
print(f'Multi Recall  \t\t', round(cls_rec * 100, 2), '%')
print(f'Multi F1_score  \t', round(cls_f1 * 100, 2), '%')

print('\r')
print(f'Hamming Loss  \t\t', round(cls_hmm * 100, 2), '%')

# print(cls_res)

Evalaution of United States of America:

All Classes (10)	 ['Earthquake', 'Volcano', 'Storm', 'Extreme Temperature', 'Flood', 'Landslide', 'Drought', 'Fire', 'Epidemic', 'Insect']

Result (EM-DAT) (5)	 ['Extreme Temperature', 'Fire', 'Flood', 'Storm', 'Earthquake']
Result (SEmHuS) (1)	 ['Storm']

Y-Act (EM-DAT) (10)	 [1, 0, 1, 1, 1, 0, 0, 1, 0, 0]
Y-Pre (SEmHuS) (10)	 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

United States of America Positive ['Severe Local Storm', 'Tropical Cyclone', 'Storm Surge']
United States of America Negative ['Heat Wave', 'Wild Fire', 'Epidemic', 'Insect Infestation', 'Snow Avalanche', 'Fire', 'Extratropical Cyclone', 'Flood', 'Tsunami', 'Mud Slide', 'Cold Wave', 'Volcano', 'Drought', 'Flash Flood', 'Earthquake', 'Land Slide']

True Positive  (1)	 ['Storm']
False Positive (0)	 []

True Negative  (5)	 ['Volcano', 'Epidemic', 'Drought', 'Insect', 'Landslide']
False Negative (4)	 ['Extreme Temperature', 'Fire', 'Flood', 'Earthquake']

Label Precision  	 100.0 %
Label Recall

In [85]:
# alpha_range = range(-12, 13)
# alpha_range = range(-1, 2)
# alpha_range = range(-8, 9)
alpha_range = [-2, 0, 2]
years_range = years

In [86]:
tot_alpha = []
tot_year = []

tot_acc = []
tot_pre = []
tot_rec = []
tot_spc = []
tot_f1 = []

tot_tp = []
tot_fp = []
tot_tn = []
tot_fn = []

all_act = []
all_pre = []

for alpha in alpha_range:

    for i in reversed(years_range):
        
        # if i==0: alpha = 0
        # else: alpha = i * 0.25

        year = i

        country = []

        true_positive = []
        false_positive = []
        true_negative = []
        false_negative = []

        tp = []
        fp = []
        tn = []
        fn = []

        alpha_act = []
        alpha_pre = []

        for plc in labPlace_tok.classes_:

            if plc not in places: continue

            SM_RE = SM_Reasons(plc)
            EM_RE = EM_Reasons(plc, year, SM_RE)

            act_sem = dis_cls(SM_RE[0])
            act_emd = EM_RE[4]

            y_act = []
            y_pre = []

            for r in New_Reason:
                if r in act_emd and r in act_sem:
                    y_act.append(1)
                    y_pre.append(1)
                elif r in act_emd and r not in act_sem:
                    y_act.append(1)
                    y_pre.append(0)
                elif r not in act_emd and r in act_sem:
                    y_act.append(0)
                    y_pre.append(1)
                else:
                    y_act.append(0)
                    y_pre.append(0)

            # for r in New_Reason:
            #     if r in act_sem and r in act_emd:
            #         y_act.append(r)
            #         y_pre.append(r)
            #     elif r in act_sem and r not in act_emd:
            #         y_act.append(r)
            #         y_pre.append('None')
            #     elif r not in act_sem and r in act_emd:
            #         y_act.append('None')
            #         y_pre.append(r)
            #     else:
            #         y_act.append('None')
            #         y_pre.append('None')

            alpha_act.append(y_act)
            alpha_pre.append(y_pre)

            num_tp = len(EM_RE[0])
            num_fp = len(EM_RE[1])
            num_tn = len(EM_RE[2])
            num_fn = len(EM_RE[3])

            num_sum = num_tp + num_fp + num_tn + num_fn
            if num_sum == 0: continue

            country.append(plc)

            true_positive.append(EM_RE[0])
            false_positive.append(EM_RE[1])
            true_negative.append(EM_RE[2])
            false_negative.append(EM_RE[3])

            tp.append(num_tp)
            fp.append(num_fp)
            tn.append(num_tn)
            fn.append(num_fn)
        
        resultDict = {'Country': country, 'True Positive': true_positive, 'False Positive': false_positive, 'True Negative': true_negative, 'False Negative': false_negative, 'TP': tp, 'FP': fp, 'TN': tn, 'FN': fn}
        resultDF = pd.DataFrame(data=resultDict)
        
        resultDF['Accuracy'] = (resultDF['TP']+resultDF['TN']) / (resultDF['TP']+resultDF['TN']+resultDF['FP']+resultDF['FN'])
        resultDF['Precision'] = (resultDF['TP']) / (resultDF['TP']+resultDF['FP'])
        resultDF['Recall'] = (resultDF['TP']) / (resultDF['TP']+resultDF['FN'])
        resultDF['F1_Score'] = 2 * (resultDF['Recall'] * resultDF['Precision']) / (resultDF['Recall'] + resultDF['Precision'])

        val_result = f'./Outputs/{model_name}Validated_Results_EM_DAT_Alpha_{alpha}_Year_{year}.csv'
        resultDF.to_csv(val_result)

        TP = resultDF['TP'].sum()
        FP = resultDF['FP'].sum()
        TN = resultDF['TN'].sum()
        FN = resultDF['FN'].sum()

        Accuracy = (TP+TN) / (TP+FP+FN+TN)
        Precision = TP / (TP+FP)
        Recall = TP / (TP+FN)
        F1_Score = 2 * (Recall * Precision) / (Recall + Precision)

        # Accuracy = round(Accuracy * 100, 2)
        # Precision = round(Precision * 100, 2)
        # Recall = round(Recall * 100, 2)
        # F1_Score = round(F1_Score * 100, 2)

        print(f'The Year \t: {year}')
        print(f'Alpha Value \t: {alpha}')
        print(f'Accuracy \t: {round(Accuracy * 100, 2)} %')
        print(f'Precision \t: {round(Precision * 100, 2)} %')
        print(f'Recall \t\t: {round(Recall * 100, 2)} %')
        print(f'F1 Score \t: {round(F1_Score * 100, 2)} %\n')

        tot_tp.append(TP)
        tot_fp.append(FP)
        tot_tn.append(TN)
        tot_fn.append(FN)

        tot_alpha.append(alpha)
        tot_year.append(year)
        tot_acc.append(Accuracy)
        tot_pre.append(Precision)
        tot_rec.append(Recall)
        tot_f1.append(F1_Score)

        all_act.append(alpha_act)
        all_pre.append(alpha_pre)

resDict = {'Alpha': tot_alpha, 'Year': tot_year, 'TP': tot_tp, 'FP': tot_fp, 'TN': tot_tn, 'FN': tot_fn, 'Accuracy': tot_acc, 'Precision': tot_pre, 'Recall': tot_rec, 'F1': tot_f1}
resDF = pd.DataFrame(data=resDict)
res_file = f'./Outputs/{model_name}Accuracy_Precision_Recall_F1_Results_per_Year.csv'
resDF.to_csv(res_file)

The Year 	: 2016
Alpha Value 	: -2
Accuracy 	: 79.81 %
Precision 	: 39.47 %
Recall 		: 24.46 %
F1 Score 	: 30.2 %

The Year 	: 2015
Alpha Value 	: -2
Accuracy 	: 79.46 %
Precision 	: 43.75 %
Recall 		: 26.17 %
F1 Score 	: 32.75 %

The Year 	: 2014
Alpha Value 	: -2
Accuracy 	: 77.7 %
Precision 	: 36.13 %
Recall 		: 22.63 %
F1 Score 	: 27.83 %

The Year 	: 2013
Alpha Value 	: -2
Accuracy 	: 80.19 %
Precision 	: 39.5 %
Recall 		: 25.41 %
F1 Score 	: 30.92 %

The Year 	: 2012
Alpha Value 	: -2
Accuracy 	: 79.91 %
Precision 	: 44.07 %
Recall 		: 25.62 %
F1 Score 	: 32.4 %

The Year 	: 2011
Alpha Value 	: -2
Accuracy 	: 79.18 %
Precision 	: 47.71 %
Recall 		: 26.4 %
F1 Score 	: 33.99 %

The Year 	: 2010
Alpha Value 	: -2
Accuracy 	: 78.42 %
Precision 	: 47.66 %
Recall 		: 24.11 %
F1 Score 	: 32.02 %

The Year 	: 2009
Alpha Value 	: -2
Accuracy 	: 75.32 %
Precision 	: 32.2 %
Recall 		: 16.38 %
F1 Score 	: 21.71 %

The Year 	: 2008
Alpha Value 	: -2
Accuracy 	: 80.34 %
Precision 	: 46.61 %
Re

In [87]:
li = []

for i in reversed(years_range):
    alpha_file = f'./Outputs/{model_name}Validated_Results_EM_DAT_Alpha_{alpha}_Year_{i}.csv'
    df = pd.read_csv(alpha_file, index_col=None, header=0)
    df['Alpha'] = np.repeat(alpha, len(df))
    df['Year'] = np.repeat(i, len(df))
    df['Accuracy'] = ((df['TP']+df['TN']) / (df['TP']+df['FP']+df['FN']+df['TN']))
    df['Precision'] = (df['TP'] / (df['TP']+df['FP']))
    df['Recall'] = (df['TP'] / (df['TP']+df['FN']))
    df['F1_Score'] = 2 * (df['Recall'] * df['Precision']) / (df['Recall'] + df['Precision'])
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
comb_file = f'./Outputs/{model_name}Validated_Results_EM_DAT_Alpha_All.csv'
frame.to_csv(comb_file)